In [2]:
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [3]:
%%notify
import os
import glob
data_dir='./data/aclImdb/'
trainX,testX, trainy,testy=[],[],[],[]
for data_type in ['train','test']:
    for sentiment in ['pos','neg']:
        path=os.path.join(data_dir,data_type,sentiment,'*.txt')
        files=glob.glob(path)
        
        for f in files:
            with open(f,encoding='utf8') as review:
                if(data_type=='train'):
                    trainX.append(review.read())
                    trainy.append(1 if sentiment=='pos' else 0)
                else:
                    testX.append(review.read())
                    testy.append(1 if sentiment=='pos' else 0)

<IPython.core.display.Javascript object>

In [4]:
print('Training data of size {}'.format(len(trainX)))
print('Testing data of size {}'.format(len(testX)))

Training data of size 25000
Testing data of size 25000


In [5]:
from sklearn.utils import shuffle

trainX,trainy=shuffle(trainX,trainy)
testX,testy=shuffle(testX,testy)

In [6]:
print('Training data of size {}'.format(len(trainX)))
print('Testing data of size {}'.format(len(testX)))

Training data of size 25000
Testing data of size 25000


In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

In [8]:
import pickle

cache_dir='./cache'
cache_file='preprocessed_data.pkl'

cache_data=None

try:
    with open(os.path.join(cache_dir,cache_file),'rb') as f:
        cache_data=pickle.load(f)
    print('Read data from cache')
except:
    print('Have to preprocess the data')
    pass

Have to preprocess the data


In [ ]:
%%notify
if cache_data is None:
    train_data=[review_to_words(review) for review in trainX]
    test_data=[review_to_words(review) for review in testX]
    
    train_label=trainy
    test_label=trainy
    
    trainX=testX=trainy=testy=None
    
    cache_data=dict(train_data=train_data,
                   test_data=test_data,
                   train_label=train_label,
                   test_label=test_label)
    
    with open(os.path.join(cache_dir,cache_file),'wb') as f:
        pickle.dump(cache_data,f)
    
else:
    train_data=cache_data['train_data']
    test_data=cache_data['test_data']
    train_label=cache_data['train_label']
    test_label=cache_data['test_label']